## Import packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib_inline import backend_inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, cross_val_score,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import PartialDependenceDisplay
backend_inline.set_matplotlib_formats('svg')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Read the dataset

In [ ]:
medical_costs=pd.read_csv('https://raw.githubusercontent.com/FlipRoboTechnologies/ML-Datasets/main/Medical%20Cost%20Insurance/medical_cost_insurance.csv')

## Exploring the data

In [ ]:
medical_costs.info()

In [ ]:
medical_costs.children=medical_costs.children.astype('object')
medical_costs.info()

## Univariate EDA

### Categorical columns

In [ ]:
cat_col_df=medical_costs.select_dtypes('object')
for col in cat_col_df.columns.values:
    print(cat_col_df[col].value_counts(normalize=True),'\n')

In [ ]:
medical_costs.children=np.where((medical_costs['children']==3) | (medical_costs['children']==4) | (medical_costs['children']==5), '3 or more', medical_costs.children)
medical_costs.children.value_counts(normalize=True)

In [ ]:
cat_col_df=medical_costs.select_dtypes('object')
for col in cat_col_df.columns.values:
    plt.figure(figsize=(10,6))
    cat_col_df[col].value_counts(normalize=True).plot.bar(stacked=True)
    plt.show()

### Numerical columns

In [ ]:
num_col_df=medical_costs.select_dtypes('number')
num_col_df.describe()

In [ ]:
for col in num_col_df.columns.values:
    plt.figure(figsize=(10,6))
    num_col_df[col].plot.hist(bins=30)
    plt.xlabel(col)
    plt.show()

## Within features bivariate EDA

In [ ]:
num_col_features_df=num_col_df.drop(columns=['charges'])
num_col_features_df.corr()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(num_col_features_df.corr(numeric_only=True),
            annot=True,
            cmap='coolwarm',
            vmax=1,
            vmin=-1
);

In [ ]:
sns.pairplot(num_col_features_df,size=4)
plt.show()

In [ ]:
for col1 in cat_col_df.columns.values:
    for col2 in cat_col_df.columns.values:
        if col1 != col2:
            print(pd.crosstab(cat_col_df[col1],cat_col_df[col2],normalize='index',margins=True),'\n')

In [ ]:
for col1 in cat_col_df.columns.values:
    for col2 in cat_col_df.columns.values:
        if col1 != col2:
            plt.figure(figsize=(10,6))
            pd.crosstab(cat_col_df[col1],cat_col_df[col2],normalize='index',margins=True).plot.bar()
            plt.show()

In [ ]:
for cat_col in cat_col_df.columns.values:
    for num_col in num_col_df.columns.values:
        if num_col!='charges':
            plt.figure(figsize=(10,6))
            medical_costs.groupby(cat_col)[num_col].mean().plot.bar()
            plt.ylabel(num_col)
            plt.show()

## Between feature and target bivariate EDA

In [ ]:
for col in medical_costs.drop(columns=['charges'],axis=1).columns.values:
    if medical_costs[col].dtypes=='object':
        plt.subplots(figsize=(12,6))
        medical_costs.groupby(col)['charges'].median().plot.bar()
        plt.axhline(y=medical_costs.charges.median(), color='red', linestyle='--')
        plt.ylabel('charges')
        plt.show()
    else:
        sns.lmplot(data=medical_costs,x=col,y='charges',fit_reg=True,height=6,aspect=1.5)
        plt.show()

## Feature engineering for model building

In [ ]:
children=pd.get_dummies(medical_costs.children,prefix='children',drop_first=True).astype('int64')
region=pd.get_dummies(medical_costs.region,prefix='region',drop_first=True).astype('int64')
sex=pd.get_dummies(medical_costs.sex,prefix='sex',drop_first=True).astype('int64')
smoker=pd.get_dummies(medical_costs.smoker,prefix='smoker',drop_first=True).astype('int64')
cat_dummy_df=pd.concat([sex,smoker,region,children],join='inner',axis=1)

In [ ]:
num_features=medical_costs.select_dtypes('number').drop(columns=['charges'],axis=1)

In [ ]:
X=pd.concat([num_features,cat_dummy_df],axis=1,join='inner')
y=medical_costs.charges

## Preprocessing

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42)

In [ ]:
scaler=MinMaxScaler()
scaler.fit(X_train)

In [ ]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

## Model building

### Finding the best random state

In [ ]:
mae_diff={}
for rs in range(1,1000):
    rf=GradientBoostingRegressor(random_state=rs)
    cv=KFold(n_splits=10,shuffle=True,random_state=rs)
    cv_mae=cross_val_score(
        estimator=rf,
        X=X_train_scaled,
        y=y_train,
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    ).mean()*-1
    # print(f'cv MAE: {cv_mae}')
    rf.fit(X_train_scaled,y_train)
    pred=rf.predict(X_test_scaled)
    test_mae=mean_absolute_error(y_true=y_test,y_pred=pred)
    # print(f'Test MAE: {test_mae}')
    mae_diff[rs]=abs(test_mae-cv_mae)
best_random_state=min(mae_diff,key=mae_diff.get)
print(f'Best random state = {best_random_state}')
print(f'Lowest MAE = {mae_diff[best_random_state]}')

## Selecting the best ML algorithm

In [ ]:
def get_model_scores(model):
    cv=KFold(n_splits=10,shuffle=True,random_state=best_random_state)
    cv_mae=cross_val_score(
        estimator=model,
        X=X_train_scaled,
        y=y_train,
        cv=cv,
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    ).mean()*-1
    model.fit(X_train_scaled,y_train)
    print(f'CV MAE = {cv_mae}')
    print(f'Test MAE = {mean_absolute_error(y_true=y_test,y_pred=model.predict(X_test_scaled))}')
    print(f'MAE difference between cv and test = {abs(mean_absolute_error(y_true=y_test,y_pred=model.predict(X_test_scaled))-cv_mae)}')
    return abs(mean_absolute_error(y_true=y_test,y_pred=model.predict(X_test_scaled))-cv_mae)

In [ ]:
ml_algo_dict={}

In [ ]:
# Linear regression
lr=LinearRegression()
ml_algo_dict[lr]=get_model_scores(lr)

In [ ]:
# Elastic net
en=ElasticNet(random_state=best_random_state)
ml_algo_dict[en]=get_model_scores(en)

In [ ]:
# Ridge
ridge=Ridge(random_state=best_random_state)
ml_algo_dict[ridge]=get_model_scores(ridge)

In [ ]:
# Lasso
lasso=Lasso(random_state=best_random_state)
ml_algo_dict[lasso]=get_model_scores(lasso)

In [ ]:
# Decission tree
dt=DecisionTreeRegressor(random_state=best_random_state)
ml_algo_dict[dt]=get_model_scores(dt)

In [ ]:
# Random forest
rf=RandomForestRegressor(random_state=best_random_state)
ml_algo_dict[rf]=get_model_scores(rf)

In [ ]:
# Support vector regression
svr=SVR()
ml_algo_dict[svr]=get_model_scores(svr)

In [ ]:
# Bagging regressor
bgr=BaggingRegressor(random_state=best_random_state)
ml_algo_dict[bgr]=get_model_scores(bgr)

In [ ]:
# Gradient boosting
gbr=GradientBoostingRegressor(random_state=best_random_state)
ml_algo_dict[gbr]=get_model_scores(gbr)

In [ ]:
selected_model=min(ml_algo_dict,key=ml_algo_dict.get)
y_pred=selected_model.predict(X_test_scaled)
mean_absolute_error(y_true=y_test,y_pred=y_pred)

## Hyperparameter tuning

In [ ]:
param_grid={
    'alpha': np.linspace(0,1000,10001),
    'fit_intercept': [True,False],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
}
cv=KFold(n_splits=10,shuffle=True,random_state=best_random_state)
model=GridSearchCV(
    estimator=selected_model,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)
model.fit(X_train_scaled,y_train)
model.best_score_

### the earlier model performed better than the best model found through hyperparameter tuning. So the earlier selected model is our final model

### Feature importance

In [ ]:
feature_importance=pd.Series(data=selected_model.coef_,index=X.columns.values)
plt.figure(figsize=(10,6))
feature_importance.sort_values().plot.barh();

In [ ]:
feature_importance

### Partial dependence plots

In [ ]:
pred_df=pd.DataFrame(columns=X.columns.values,
                     data=scaler.transform(X))
for col in pred_df.columns.values:
    fig,ax=plt.subplots(figsize=(10, 6))
    plt.figure(figsize=(10,6))
    pdp = PartialDependenceDisplay.from_estimator(selected_model,
                                                  pred_df,
                                                  [col],
                                                  kind='average',
                                                  ax=ax
                                                  );

## Insights

#### From the feature importance plot above it is evident that smoking, bmi and age are the top three factors with smoking being the most important in determining the medical insurance claim cost to the company.

Based on the results of the above analyses, we make the below observations.

* Those who smoke are very highly likely to make a high claim for medical related cost as compared to those who don't smoke in general. As an estimate those who smoke are likely to claim $23464 more than those who don't smoke on average.
* Similarly, people with higher bmi are more likely to make higher claim. To be precise someone who is one unit higher in bmi is likely to claim $12058 more than someone who is one unit below in bmi than him/her.
* Likewise, older people are more likely to make higher claim than younger people which is normal as older people are more susceptible to fall sick thus incurring higher medical cost. Precisely, with each year in age people are likely to claim $11733 more compared to someone who is a year younger on average.
* Furthermore, people with children are likely to make higher claim compared to people with no children.
* Finally, people from northeast region are likely to make slightly higher claim compared to people from other region in general. However, this factor may not be significantly important to consider for making any business decision.

Hence, one recommendation that could be made out of these findings is that the insurance company may want to charge higher health insurance premium to those who are smoker, older, has bmi on the higher side, has children and live in the northeast region as people with this profile incur risk of high insurance loss as compared to others.